<a href="https://colab.research.google.com/github/roshantac/EVA4/blob/master/EVA4_Session_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv0 = nn.Conv2d(1,4,3)
        self.conv1 = nn.Conv2d(4, 8, 3) #26
        self.BN0 = nn.BatchNorm2d(4)
        self.BN1 = nn.BatchNorm2d(8)
        self.BN2 = nn.BatchNorm2d(16)
        self.BN3 = nn.BatchNorm2d(28)
        self.BN4 = nn.BatchNorm2d(56)
        self.dropout = nn.Dropout2d(.2)
        self.conv2 = nn.Conv2d(8, 16, 3)#24
        self.pool1 = nn.MaxPool2d(2, 2)# 12
        self.conv3 = nn.Conv2d(16, 28, 3)#10
        self.conv4 = nn.Conv2d(28, 56, 3)#8
        self.pool2 = nn.MaxPool2d(2, 2)# 4
        self.Avgpool = nn.AvgPool2d(3) 
        self.conv5 = nn.Conv2d(56,10,1)
        
        
    def forward(self, x):
      x = self.dropout(self.BN0(F.relu(self.conv0(x)))) #26
      x = self.dropout(self.BN1((F.relu(self.conv1(x))))) #24
      x = self.dropout(self.BN2((F.relu(self.conv2(x))))) #22
      x = self.pool2(x) # 11
      x = self.dropout(self.BN3((F.relu(self.conv3(x))))) # 9
      x = self.dropout(self.BN4((F.relu(self.conv4(x))))) # 7
      x = self.dropout(self.pool2(x)) #3 
      x = self.Avgpool(x) #
      x = self.conv5(x) 
      x = x.view(-1, 10)
      return F.log_softmax(x)


        

In [3]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 4, 26, 26]              40
       BatchNorm2d-2            [-1, 4, 26, 26]               8
         Dropout2d-3            [-1, 4, 26, 26]               0
            Conv2d-4            [-1, 8, 24, 24]             296
       BatchNorm2d-5            [-1, 8, 24, 24]              16
         Dropout2d-6            [-1, 8, 24, 24]               0
            Conv2d-7           [-1, 16, 22, 22]           1,168
       BatchNorm2d-8           [-1, 16, 22, 22]              32
         Dropout2d-9           [-1, 16, 22, 22]               0
        MaxPool2d-10           [-1, 16, 11, 11]               0
           Conv2d-11             [-1, 28, 9, 9]           4,060
      BatchNorm2d-12             [-1, 28, 9, 9]              56
        Dropout2d-13             [-1, 28, 9, 9]               0
           Conv2d-14             [-1, 5

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:32: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [0]:


torch.manual_seed(1)
batch_size = 72

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [0]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [0]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [9]:

model = Net().to(device)
optimizer = optim.Adam(model.parameters(), lr= 0.001, betas=(.9,0.999))
  # optim.SGD(model.parameters(), lr=0.01, momentum=0.999) #

for epoch in range(0, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    print(epoch)

  0%|          | 0/834 [00:00<?, ?it/s]


Test set: Average loss: 0.3200, Accuracy: 9405/10000 (94.05%)

0


  0%|          | 0/834 [00:00<?, ?it/s]


Test set: Average loss: 0.1312, Accuracy: 9693/10000 (96.93%)

1


  0%|          | 0/834 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0268, Accuracy: 9922/10000 (99.22%)

17


  0%|          | 0/834 [00:00<?, ?it/s]


Test set: Average loss: 0.0241, Accuracy: 9931/10000 (99.31%)

18


loss=0.025422891601920128 batch_id=833: 100%|██████████| 834/834 [00:16<00:00, 47.69it/s]



Test set: Average loss: 0.0248, Accuracy: 9927/10000 (99.27%)

19
